### Install Mediapipe(세션 재시작 필요)

In [1]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled pro

### Set Random Seed

In [14]:
import os
import random
import numpy as np
import tensorflow as tf

# 시드값 설정
SEED = 42

def set_global_seed(seed):
    # 1. Python 기본 시드 고정
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)

    # 2. NumPy 시드 고정
    np.random.seed(seed)

    # 3. TensorFlow(Keras) 시드 고정
    tf.random.set_seed(seed)

    # 4. TensorFlow 연산 결정론적(Deterministic) 설정 (TF 2.7+ 버전 권장)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' # GPU 사용 시

    try:
        tf.config.experimental.enable_op_determinism()
    except AttributeError:
        print("경고: 설치된 TensorFlow 버전에서 enable_op_determinism을 지원하지 않을 수 있습니다.")

### Helper Func

In [2]:
def calculate_angle(a, b, c):
    """
    세 개의 랜드마크(a, b, c)를 받아 b 지점의 각도와 신뢰도(가시성 곱)를 반환합니다.
    """
    reliability = a[3] * b[3] * c[3]

    if a[3] < 0.5 or b[3] < 0.5 or c[3] < 0.5:
        return None, reliability

    a_xyz = a[:3]
    b_xyz = b[:3]
    c_xyz = c[:3]

    try:
        ba = a_xyz - b_xyz
        bc = c_xyz - b_xyz

        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
        angle = np.arccos(cosine_angle)

        return np.degrees(angle), reliability
    except (ZeroDivisionError, RuntimeWarning):
        return None, reliability

def calculate_plane_normal(a, b, c):
    """
    세 개의 랜드마크(a, b, c)로 정의된 평면의 법선 벡터(Normal Vector)를 반환합니다.
    반환값: [x, y, z] (정규화된 벡터), reliability
    순서는 a -> b -> c 순으로 오른손 법칙을 따릅니다.
    """
    # 신뢰도 계산
    reliability = a[3] * b[3] * c[3]

    # 가시성이 낮은 랜드마크가 포함되면 계산 불가
    if a[3] < 0.5 or b[3] < 0.5 or c[3] < 0.5:
        return None, reliability

    a_xyz = a[:3]
    b_xyz = b[:3]
    c_xyz = c[:3]

    try:
        # 두 개의 벡터 생성 (a -> b, a -> c)
        v1 = b_xyz - a_xyz
        v2 = c_xyz - a_xyz

        # 외적(Cross Product)을 통해 법선 벡터 계산
        normal = np.cross(v1, v2)

        # 벡터 정규화 (크기를 1로 만듦)
        norm_length = np.linalg.norm(normal)
        if norm_length == 0:
            return None, reliability

        normalized_normal = normal / norm_length

        return normalized_normal, reliability

    except (RuntimeWarning, Exception):
        return None, reliability


## Feature Engineering

In [ ]:
def feature_engineering(landmarks):
    """
    landmarks: (33, 4) shape의 numpy array [x, y, z, visibility]
    """
    if landmarks is None: return None
    NOSE = 0
    L_EYE, R_EYE = 2, 5
    L_EAR, R_EAR = 7, 8
    L_SHOULDER, R_SHOULDER = 11, 12
    L_ELBOW, R_ELBOW = 13, 14
    L_WRIST, R_WRIST = 15, 16
    L_HIP, R_HIP = 23, 24

    normal_features, normal_rels = [], []

    face_normal, face_rel = calculate_plane_normal(landmarks[NOSE], landmarks[L_EYE], landmarks[R_EYE])
    normal_features.append(face_normal)
    normal_rels.append(face_rel)

    sh_nose_normal, sh_nose_rel = calculate_plane_normal(landmarks[L_SHOULDER], landmarks[R_SHOULDER], landmarks[NOSE])
    normal_features.append(sh_nose_normal)
    normal_rels.append(sh_nose_rel)

    flat_normals_vec = []
    for vec in normal_features:
        if vec is not None: flat_normals_vec.extend(vec)
        else: flat_normals_vec.extend([0.0, 0.0, 0.0])

    final_features = flat_normals_vec + normal_rels
    return np.array(final_features)

## 데이터셋 전처리

In [4]:
# .npy 파일 경로
X_DATA_PATH = 'X_train.npy'
Y_DATA_PATH = 'Y_train.npy'
X_VAL_PATH = 'X_val.npy'
Y_VAL_PATH = 'Y_val.npy'
X_TEST_PATH = 'X_test.npy'
Y_TEST_PATH = 'Y_test.npy'

required_files = [
    X_DATA_PATH, Y_DATA_PATH,
    X_VAL_PATH, Y_VAL_PATH,
    X_TEST_PATH, Y_TEST_PATH
]

if not all(os.path.exists(path) for path in required_files):
    print("="*50)
    print("데이터셋 파일이 스크립트와 동일한 디렉토리에 있는지 확인해주세요.")
    print("="*50)

try:
    raw_landmarks = np.load(X_DATA_PATH)
    labels = np.load(Y_DATA_PATH)
    raw_val = np.load(X_VAL_PATH)
    labels_val = np.load(Y_VAL_PATH)
    raw_test = np.load(X_TEST_PATH)
    labels_test = np.load(Y_TEST_PATH)

    # 데이터 처리 함수
    def process_data(raw_x, raw_y):
        processed_x, processed_y = [], []
        for i, lm in enumerate(raw_x):
            fv = feature_engineering(lm)
            if fv is not None:
                processed_x.append(fv)
                processed_y.append(raw_y[i])
        return np.array(processed_x), np.array(processed_y)

    print("데이터 전처리 중...")
    X_train, y_train = process_data(raw_landmarks, labels)
    X_val, y_val = process_data(raw_val, labels_val)
    X_test, y_test = process_data(raw_test, labels_test)

except Exception as e:
    print(f"오류 발생: {e}")

데이터 전처리 중...


## MLP(val_accuracy = 0.9613)

In [ ]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

def build_model(input_shape):
    model = Sequential([
        Input(shape=(input_shape,)),
        # kernel_initializer에도 시드를 명시하면 더 안전함
        Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        # Dropout에 seed 인자 추가 (중요!)
        Dropout(0.3, seed=SEED),

        Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED)),
        Dropout(0.3, seed=SEED),

        Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=SEED))
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])
    return model

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    try:
        if X_train.shape[0] == 0:
            print("오류: 모든 샘플에서 특징 공학(각도 계산 등)에 실패했습니다.")
            print("데이터셋의 가시성(visibility) 값을 확인해주세요.")
        else:

            print(f"학습 데이터: {X_train.shape[0]}개 / 테스트 데이터: {X_test.shape[0]}개")
            set_global_seed(SEED)

            # --- 4. 모델 학습 ---
            # 1. 인덱스 배열 생성 (0부터 N-1까지)
            indices = np.arange(X_train.shape[0])

            # 2. 인덱스 무작위 섞기
            np.random.shuffle(indices)

            # 3. 섞인 인덱스로 X와 y 재배열 (짝 유지됨)
            X_train = X_train[indices]
            y_train = y_train[indices]

            model = build_model(input_shape=X_train.shape[1])
            model.summary()

            print("\n모델 학습 시작...")
            history = model.fit(
                X_train,
                y_train,
                epochs=500,
                batch_size=16,
                validation_data=(X_val, y_val),
                shuffle=False,
                callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)]
            )

            print("모델 학습 완료.")

            loss, accuracy = model.evaluate(X_val, y_val)

            print(f"\n검증 데이터 평가 결과:")
            print(f"Loss (손실): {loss:.4f}")
            print(f"Accuracy (정확도): {accuracy:.4f}")

            # --- 6. 모델 저장 ---
            model.save("plane_pose_classifier_keras.h5")
            print("학습된 모델을 'plane_pose_classifier_keras.h5'로 저장했습니다.")

    except Exception as e:
        print(f"오류: 데이터셋 로드 또는 처리 중 예외 발생: {e}")




학습 데이터: 2139개 / 테스트 데이터: 250개


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_45 (Dense)                │ (None, 64)             │         9,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,905 (46.50 KB)

 Trainable params: 11,905 (46.50 KB)

 Non-trainable params: 0 (0.00 B)


모델 학습 시작...
Epoch 1/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5703 - loss: 0.7554 - val_accuracy: 0.5189 - val_loss: 0.7173
Epoch 2/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6635 - loss: 0.5885 - val_accuracy: 0.5236 - val_loss: 0.6415
Epoch 3/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7186 - loss: 0.4968 - val_accuracy: 0.5236 - val_loss: 0.6972
Epoch 4/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7648 - loss: 0.4426 - val_accuracy: 0.5991 - val_loss: 0.5271
Epoch 5/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7629 - loss: 0.4251 - val_accuracy: 0.6226 - val_loss: 0.4698
Epoch 6/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8226 - loss: 0.3657 - val_accuracy: 0.5425 - val_loss: 0.8731
Epoch 7/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8297 - loss: 0.3606 - val_accuracy: 0.9292 - val_loss: 0.2338
Epoch 8/500
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8726 - loss: 0.31


검증 데이터 평가 결과:
Loss (손실): 0.2293
Accuracy (정확도): 0.9340
학습된 모델을 'All_pose_classifier_keras.h5'로 저장했습니다.


## RandomForest (val_accuracy = 0.8688)

In [ ]:
import numpy as np
import os
import joblib  # scikit-learn 모델 저장용
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    try:
        set_global_seed(SEED)
        # --- 셔플링 (Shuffling) ---
        print("데이터 셔플링 중...")
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_train = X_train[indices]
        y_train = y_train[indices]

        # --- Random Forest 모델 학습 ---
        print(f"\n[Random Forest] 학습 시작 (데이터 개수: {len(X_train)}개)...")
        # n_estimators: 트리의 개수 (보통 100~500)
        rf_model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
        rf_model.fit(X_train, y_train)
        print("학습 완료.")

        # --- 모델 검증 ---
        y_pred = rf_model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)

        print(f"\n검증 데이터 평가 결과:")
        print(f"Accuracy (정확도): {accuracy:.4f}")
        print("\n분류 보고서:")
        print(classification_report(y_val, y_pred))

        # --- 모델 저장 ---
        joblib.dump(rf_model, "plane_pose_classifier_rf.pkl")
        print("모델 저장 완료: 'plane_pose_classifier_rf.pkl'")

    except Exception as e:
        print(f"오류 발생: {e}")

데이터 셔플링 중...

[Random Forest] 학습 시작 (데이터 개수: 2139개)...
학습 완료.

검증 데이터 평가 결과:
Accuracy (정확도): 1.0000

분류 보고서:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       110
           1       1.00      1.00      1.00       102

    accuracy                           1.00       212
   macro avg       1.00      1.00      1.00       212
weighted avg       1.00      1.00      1.00       212

모델 저장 완료: 'All_pose_classifier_rf.pkl'


## Support Vector Machine (val_accuracy = 0.9210)

In [ ]:
import os
import joblib
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    try:
        set_global_seed(SEED)
        print(f"특징 벡터 차원: {X_train.shape[1]}")

        # --- 셔플링 (Shuffling) ---
        print("데이터 셔플링 중...")
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_train = X_train[indices]
        y_train = y_train[indices]

        # --- SVM 모델 정의 ---
        print(f"\n[SVM] 학습 준비 (데이터 개수: {len(X_train)}개)...")

        # SVM 파이프라인 (스케일링 + SVM)
        svm_model = make_pipeline(
            StandardScaler(),
            SVC(kernel='rbf', C=1.0, probability=True)
        )

        # --- 1. 교차 검증 (Validation) ---
        print("교차 검증(Cross Validation) 진행 중...")
        val_scores = cross_val_score(svm_model, X_train, y_train, cv=5)
        print(f" >> 5-Fold 검증 정확도: {val_scores}")
        print(f" >> 평균 검증 정확도: {np.mean(val_scores):.4f}")

        # --- 2. 전체 데이터로 최종 학습 ---
        print("최종 모델 학습 중...")
        svm_model.fit(X_train, y_train)

        # --- 모델 검증 ---
        y_pred = svm_model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)

        print(f"\n검증 데이터 평가 결과:")
        print(f"Accuracy (정확도): {accuracy:.4f}")
        print("\n분류 보고서:")
        print(classification_report(y_val, y_pred))

        # --- 모델 저장 ---
        joblib.dump(svm_model, "plane_pose_classifier_svm.pkl")
        print("모델 저장 완료: 'plane_pose_classifier_svm.pkl'")

    except Exception as e:
        print(f"오류 발생: {e}")

특징 벡터 차원: 152
데이터 셔플링 중...

[SVM] 학습 준비 (데이터 개수: 2139개)...
교차 검증(Cross Validation) 진행 중...
 >> 5-Fold 검증 정확도: [0.92990654 0.94392523 0.95327103 0.94392523 0.94847775]
 >> 평균 검증 정확도: 0.9439
최종 모델 학습 중...

검증 데이터 평가 결과:
Accuracy (정확도): 0.8726

분류 보고서:
              precision    recall  f1-score   support

           0       0.80      1.00      0.89       110
           1       1.00      0.74      0.85       102

    accuracy                           0.87       212
   macro avg       0.90      0.87      0.87       212
weighted avg       0.90      0.87      0.87       212

모델 저장 완료: 'All_pose_classifier_svm.pkl'


## Log_Regression(val_accuracy = 0.5708)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import mediapipe as mp
import cv2

set_global_seed(SEED)
scaler = StandardScaler()
print("데이터 셔플링 중...")
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
y_train = y_train[indices]
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

log_reg_model = LogisticRegression(solver='liblinear')
log_reg_model.fit(X_train_scaled, y_train)

print("✅ 모델 학습 및 환경 준비 완료")

# --- 2. 모델 검증 코드 ---
print("\n--- 🧪 모델 검증 결과 (Test Set) ---")
y_pred = log_reg_model.predict(X_val_scaled)

# 정확도 (Accuracy)
accuracy = accuracy_score(y_val, y_pred)
print(f"✅ 정확도: {accuracy * 100:.2f}%")

# 혼동 행렬 (Confusion Matrix)
print("\n[ 혼동 행렬 (Confusion Matrix) ]")
print(confusion_matrix(y_val, y_pred))

# 분류 리포트 (Precision, Recall, F1-score)
# target_names는 0번 클래스와 1번 클래스의 이름을 지정합니다.
print("\n[ 분류 리포트 (Classification Report) ]")
print(classification_report(y_val, y_pred, target_names=['Bad Posture (0)', 'Good Posture (1)']))
print("-----------------------------------")

✅ 모델 학습 및 환경 준비 완료

--- 🧪 모델 검증 결과 (Test Set) ---
✅ 정확도: 94.81%

[ 혼동 행렬 (Confusion Matrix) ]
[[110   0]
 [ 11  91]]

[ 분류 리포트 (Classification Report) ]
                  precision    recall  f1-score   support

 Bad Posture (0)       0.91      1.00      0.95       110
Good Posture (1)       1.00      0.89      0.94       102

        accuracy                           0.95       212
       macro avg       0.95      0.95      0.95       212
    weighted avg       0.95      0.95      0.95       212

-----------------------------------
